In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `~/Study/algo-trading-notes`


In [2]:
using YFinance
using Dates
using CSV
using RollingFunctions
using DataFrames
using StatsBase

In [3]:
function get_stock_prices(ticker::String, startdt::String, enddt::String, interval::String)
    data_ordered_dict = get_prices(ticker, startdt=startdt, enddt=enddt, interval=interval)
    df = DataFrame(data_ordered_dict)
    df[!, :SMA1] = vcat(fill(NaN, 41), rollmean(df[!, :close], 42))
    df[!, :SMA2] = vcat(fill(NaN, 251), rollmean(df[!, :close], 252))
    df[!, :return] = vcat(fill(NaN, 1), diff(df[!, :close]) ./ df[!, :close][1:end-1])
    return df
end

get_stock_prices (generic function with 1 method)

In [4]:
data = get_stock_prices("AAPL", "2020-01-01", "2024-01-01", "1d")

last(data, 10)

Row,ticker,timestamp,open,high,low,close,adjclose,vol,SMA1,SMA2,return
,String,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AAPL,2023-12-15T14:30:00,197.53,198.4,197.0,197.57,197.57,1.28257e8,184.745,169.999,-0.00272572
2,AAPL,2023-12-18T14:30:00,196.09,196.63,194.39,195.89,195.89,5.57519e7,185.222,170.235,-0.00850336
3,AAPL,2023-12-19T14:30:00,196.16,196.95,195.89,196.94,196.94,4.07141e7,185.734,170.483,0.00536017
4,AAPL,2023-12-20T14:30:00,196.9,197.68,194.83,194.83,194.83,5.22428e7,186.256,170.731,-0.0107139
5,AAPL,2023-12-21T14:30:00,196.1,197.08,193.5,194.68,194.68,4.64825e7,186.772,170.978,-0.000769949
6,AAPL,2023-12-22T14:30:00,195.18,195.41,192.97,193.6,193.6,3.71228e7,187.252,171.209,-0.0055475
7,AAPL,2023-12-26T14:30:00,193.61,193.89,192.83,193.05,193.05,2.89193e7,187.775,171.45,-0.00284092
8,AAPL,2023-12-27T14:30:00,192.49,193.5,191.09,193.15,193.15,4.80877e7,188.4,171.693,0.000517953
9,AAPL,2023-12-28T14:30:00,194.14,194.66,193.17,193.58,193.58,3.40499e7,189.004,171.946,0.00222629


In [26]:
using DataFrames

# Create a sample DataFrame
df = DataFrame(A = 1:5, B = 6:10)

# Shift the column manually
df.A_shifted = [missing; df.A[1:end-1]]  # Shift down by 1

# Calculate the rate of change
# The result for the first row will be missing due to the shift
df.rate_of_change = (df.A - df.A_shifted) ./ df.A_shifted

# Optionally, handle missing values
# For instance, replace them with a default value
df.rate_of_change = coalesce.(df.rate_of_change, 0) # Replacing missing with 0


5-element Vector{Real}:
 0
 1.0
 0.5
 0.3333333333333333
 0.25

In [ ]:
# do a value count of position
countmap(data_filled[!, :position])

In [ ]:
# get last element of cum_strategy_return
data_filled[end, :cum_strategy_return] - data_filled[end, :cum_return]

In [22]:
# test a range of sma1 and sma2 values to minimize the difference
sma1_range = 10:10:100
sma2_range = 200:10:300
results = DataFrame(sma1=[], sma2=[], diff=[])
for sma1 in sma1_range
    for sma2 in sma2_range
        data = get_stock_prices("AAPL", "2020-01-01", "2024-01-01", "1d")
        output = backtest_strategy(data[.!isnan.(data[:, :SMA2]), :])
        diff = output[1]
        push!(results, [sma1, sma2, diff])
    end
end

In [19]:
using Optim

# Define a function that calculates the difference for given sma1 and sma2
function calculate_difference(sma)
    sma1, sma2 = sma
    data = get_stock_prices("AAPL", "2020-01-01", "2024-01-01", "1d")
    data = backtest_strategy(data[.!isnan.(data[:, :return]), :])
    output = backtest_strategy(data[.!isnan.(data[:, :return]), :])[0]
    return output
end

# Initial guess
initial_sma = [50.0, 250.0]

# Perform optimization
opt_result = optimize(calculate_difference, initial_sma, BFGS())

# Get the optimized parameters
optimized_sma = Optim.minimizer(opt_result)

MethodError: MethodError: no method matching getindex(::Tuple{Float64, Float64}, ::Colon, ::Symbol)

Closest candidates are:
  getindex(::Tuple, ::Colon)
   @ Base tuple.jl:33
  getindex(::Tuple, !Matched::Int64)
   @ Base tuple.jl:29
  getindex(::Tuple, !Matched::Integer)
   @ Base tuple.jl:30
  ...
